# Old code for new results

Last updated: 20 September 2020

In [ ]:
# Do evolved calc yield
def evolved_calc_yield(dtm=None, m = None, b = None):
    yield_kg_ha = m*dtm + b

    if dtm > 185:
        raise ValueError("days to maturity, {dtm} is larger than 175".format(
                dtm=dtm))
    if dtm < 68:
        raise ValueError("days to maturity, {dtm} is less than 68".format(
                dtm=dtm))

    return yield_kg_ha

## Part 0. Account for different maximum yields
# verified using Kenya Seed Co. - https://web.archive.org/web/20190819125927/http://kenyaseed.com/gallery/maize/
verified_hybrid_data = pd.read_csv('../data/Yields/hybrid_yields_verified.csv')

# convert to metric tons
verified_hybrid_data['yield_metric_tons'] = verified_hybrid_data.verified_yield_kg_acre/1000
p, m, b = plot_lin_regression(verified_hybrid_data.verified_days_to_maturity, verified_hybrid_data.yield_metric_tons, 
                             'verified_days_to_maturity', 'yield_metric_tons', verified_hybrid_data, 
                             85, 4.9, 'Days to Maturity (days)', 'Yield (tons/ha)', 
                             'Potential Maize Yields from Kenya Seed Company', positive=False)

#plt.savefig('../output/6-results/ksc.png',format='png')

In [ ]:
def model_runs(n_sim = 1000, burn_in = 60, station = 'OL JOGI FARM', texture = 'clay loam', lgp=180, pd_sim=60): 
    """ Run simulations for a specific climate, soil, and crop.

    Usage: total_rf_corr(n_sim, pd_ic, station, texture, lgp, pd_sim)
           
    Default values:
        n_sim = 1000 # Number of Simulations to Remove Initial Conditions [integer]
        pd_ic = 1 # Planting Date for Initial Conditions [Julian day]
        station = 'OL JOGI FARM' # Climatology [string]
        texture = 'clay' # Soil Type [string]
        lgp = 180 # Length of Growing Period [Days]
        pd_sim 100 # Planting Date for Simulation [Julian day]

    """
    # Part 1. Set conditions for IC runs
    doy = abs(burn_in - pd_sim) # absolute value

    # Part 2. Initialize model with a climate, soil and crop
    climate = Climate(station=station)
    soil = Soil(texture=texture)
    crop = Crop(soil=soil, lgp=lgp)
    soil.set_nZr(crop)

    model = CropModel(crop=crop, climate=climate, soil=soil)
    model.run()
    o = model.output()

    # Part 3. Get the mean, SD soil moisture and run the simulations to remove IC
    s0_mean, s0_std = average_soil_moisture(model, n_sims=n_sim, doy=doy)
    models = [CropModel(crop=crop, climate=Climate(), soil=soil) for i in np.arange(n_sim)]
    
    # Part 4. Run the actual simulations
    planting_date = pd_sim
    output = [model.run(s0=s0_mean, do_output=True, planting_date=planting_date) for model in models]

    # Part 5. Subset the growing period and get avg stress, total rf
    start = burn_in 
    end = start + lgp
    avg_stress = [df[start:end]['stress'].mean() for df in output]
    total_stress = [df[start:end]['stress'].sum() for df in output]

    # dont need this but helpful for debugging
    total_rf = [df[start:end]['R'].sum() for df in output]
    avg_s = [df[start:end]['s'].mean() for df in output]
    sat = [df[start:end]['s'] for df in output]
    stress = [df[start:end]['stress'] for df in output]
    
    o = output
    
    # Part 6. Output the dynamic stress and yield values
    # prev crop.calc_dstress
    x = [crop.calc_dstress(s=df.s, stress=df.stress) for df in o] #for i in np.arange(n_sim)
    #print(x)
    
    # Part 7. Define kc
    start = burn_in
    end = burn_in + lgp
    kc = o[1][start:end]['kc'].tolist()
    
    return  x, total_rf, sat, stress, o, crop, output, kc, start, end #output, avg_stress, total_stress, total_rf, mstr_memb, dstr_memb, yield_kg_ha

# Note: Not great practice to return all of these variables, but for testing it's useful
data, rf, sat, stress, o, crop, output, kc, start, end = model_runs()

In [ ]:
# I don't actually use these plots in the MS
# Do scatter plot with three variety types

f, axs = plot.subplots(nrows=1, ncols=3, journal='agu2', spanx=True, tight=True) #, tight=False

result['Yield_norm'] = result.Yield/max(result.Yield)
datas = ["Category == 'Early (< 105 d)'", "Category == 'Medium (110 - 140 d)'", "Category == 'Late (145 + d)'"]

# one option for colors
#import matplotlib.cm as cm
#colors = cm.rainbow(np.linspace(0, 1, 3))

# another option for colors
red = sns.color_palette("reds")[-2] 
yellow = sns.color_palette("orange1")[-3]
blue = sns.color_palette("blues")[-1]
colors = [red, yellow, blue]

for ax, data, c in zip(axs, datas, colors):
    dat = result.query(data)
    ax.scatter(dat.RF, dat.Yield_norm, color=c)

axs.format(xlim=(0, 900), ylim=(-0.05, 1.05), 
    abc=True, xlabel='Rainfall (mm)', ylabel='Yield normalized by max. yield', abcloc='ul', 
           collabels=['Early', 'Medium', 'Late'],
           suptitle='Rainfall-Yield Relationship Using Dynamic Stress (k=0.2, r=0.5)')

#plt.savefig('../output/figures/yieldrf.png',format='png')


In [ ]:
# make sure this starts on the day that we want
output[start:end]
# output[0][60:240]

# # Troubleshooting Crop Coefficient. Move this elswhere
# o = output[0]
# o.index = o.index+1 # make doy cooperate better
# plt.plot(o['kc'], 'r-') 
# #plt.set(ylabel='Crop coefficient, $\mathit{kc}$')
# #plt.set_ylim(0.2, 1.4)
# #plt.legend(['kc'],frameon=False)

In [ ]:
# output[1]

# # TODO, Output a text file with the model parameters

# # Just messing around with stuff
# for i in no_sims:
#     #print(output[i][start:end]['s'].min())
#     _s_min = output[i]['L'].max()
#     s_min.append(_s_min)
# result = pd.DataFrame({'L_max':s_min})


In [ ]:
# figure set up
# nice figure that should go in the paper
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3,ncols=1, sharex=False)
#fig, axs = plot.subplots(nrows=3, ncols=1, sharex=False)
left   =  0.135  # the left side of the subplots of the figure
right  =  0.9    # the right side of the subplots of the figure
bottom =  0.1    # the bottom of the subplots of the figure
top    =  0.9    # the top of the subplots of the figure
wspace =  .3     # the amount of width reserved for blank space between subplots
hspace =  0.5    # the amount of height reserved for white space between subplots

# This function actually adjusts the sub plots using the above paramters
plt.subplots_adjust(
    left    =  left, 
    bottom  =  bottom, 
    right   =  right, 
    top     =  top, 
    wspace  =  wspace, 
    hspace  =  hspace
)
#axs.format(abc=True, abcloc='ul', suptitle='Non-stationarity in Model Parameters')


# The amount of space above titles
#y_title_margin = 1.2

# Plot a) Rainfall climatology
# I'm not sure that I need anything from above

alpha_monthly, lambda_monthly = make_climate_parameters()
# make a dataframe
df = pd.DataFrame({'alpha':alpha_monthly, 'lambda':lambda_monthly, 'month':np.arange(1,13,1),'month_A':['J','F','M','A',
                                                                           'M','J','J','A','S','O','N','D']})
# now create a longer timeseries using month three to eight: March 1st and 1980 days later.

#subset dataframe
#ax1=axs[0]
df['doy'] = df[2:8]['month']*30
df = df[2:8].set_index('month_A').drop(['month','doy'], axis=1)

df.plot(kind="bar", ax=ax1, ylim=(0,18),rot=360, color=['lightgray','darkgray']) # hatch=('-','//')
ax1_ = ax1.twinx()
for r in ax1.patches[len(df):]:
    r.set_transform(ax1_.transData)

ax1.set_ylabel(r'Rain per storm, $\alpha$ (mm)')
ax1_.set_ylabel('Inter-storm arrival, $\lambda$ (day$^{-1}$)')
#ax1.set_xlabel([], color='white')
ax1.set_xlabel('Month')
ax1_.set_xlabel([''])

ax1.legend(frameon=False, ncol=2)
ax1.grid(which='major',axis='both')


# Plot b) Crop Coefficient
o = output[0]
o.index = o.index+1 # make doy cooperate better
ax2.plot(o['kc'], 'r-') 
ax2.set(ylabel='Crop coefficient, $\mathit{kc}$')
ax2.set_ylim(0.2, 1.4)
ax2.legend(['kc'],frameon=False)

# Plot c) Saturation
# get average saturation
avg_df_180 = pd.concat([df['s'] for df in output], axis=1)                          
avg_df_180["mean"] = avg_df_180.mean(axis=1)  
avg_df_180["var"] = avg_df_180.var(axis=1)  
df_180 = avg_df_180
df_180 = df_180[0:180] ##############!!!!!!!!!!!!!!!! something is goofy here, come back and take a look
df_180.index = df_180.index+1 # make doy cooperate better

ax3.plot(df_180['mean'], 'k-')
x = np.arange(1, len(df_180)+1, 1)
ax3.fill_between(x, df_180['var']+df_180['mean'], df_180['mean']-df_180['var'],facecolor='lightblue') #, facecolor='lightsalmon'
ax3.set(xlabel='Day of growing season, $\mathit{d}$')
ax3.legend(['Mean', 'Variance'], ncol=2, frameon=False)
ax3.set(ylabel='Relative soil \n moisture content, $\mathit{s}$')
plt.savefig('../output/6-results/threefigs.png',format='png')


# Old code for old results

In [1]:
# Everything under here last updated 13 April 2020

# import packages and set working directory
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from math import exp
import pandas as pd

# We need to add the module path to our system path so 
# so that our notebook can find our local objects and code:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# os.chdir("../")

# import objects
from farm import Climate
from farm import Soil
from farm import Crop
from farm import CropModel
from farm.functions import make_climate_parameters, average_soil_moisture

ImportError: cannot import name 'make_climate_parameters' from 'farm.functions' (/Users/charrua/Box Sync/waves/maize-Toff/farm/functions.py)

## Finding the initial conditions

Why are we doing this? For each soil type and climatology, find the initial soil moisture value that is basically the value the time series converges to when you run many simulations.

These are the notes on what to do based on 19 February meeting with Kelly and Bryn

   * **Step 0.** Decide on soil type and climate; in this example: loam, Ol jogi.
   * **Step 1.** Start DOY before the planting season. Pick a day when there isn’t much rain: Jan 1. 
   * **Step 2.** Run t_sim for 365 so the calendar ends on Dec. 31. 
   * **Step 3.** Start planting date on Jan 21.
   * **Step 4.** Return just the soil moisture in a dataframe and append the simulation number to x
   * **Step 5.** Re-run the simulation where the soil moisture on Jan 1 is the value that ended on Dec. 31. 
   * **Step 6.** Run this lots and lots of times. 
   * **Step 7.** Remove the first year of data because that was an artificial soil moisture value. 
   * **Step 8.** End goal: For a given day of the year, return the mean and variance of the soil moisture for each day. Plot the results.
   
   
How to make this work for 365 days.. If I want to start the model on a day there isn't much rain, January 1st, use 21 as `planting_date` and `t_before` as 21. If I want to model to end on December 31, then need `t_after` is: 21 + 180 - 365 = 164

In [ ]:
output

In [ ]:
model = CropModel(crop=crop,soil=soil,climate=climates[1])
model.climate.

In [ ]:
model.run()
self = model
range(self.n_days)

In [ ]:
# We only need to set the climate once. 
alpha_r, lambda_r = make_climate_parameters()

def initial_conditions(simulations, sm0=0.3):
    
    # generate little progress bar
    import time
    import progressbar
    
    for i in progressbar.progressbar(range(simulations)):
        time.sleep(0.02)
    
    df = pd.DataFrame()
    
    # run this many times
    for i in range(simulations):

        climate = Climate(alpha_r, lambda_r)
        soil = Soil('loam')
        crop = Crop(soil=soil)
        soil.set_nZr(crop)  
        model = CropModel(crop=crop,soil=soil,climate=climate)
        
        # Step 1.  Start DOY before the planting season, Jan 1. 
        # Step 2. Run t_sim for 365 so that calendar ends on Dec. 31
        # Step 3. Start planting date on some reasonable planting date
        model.run(s0=sm0, planting_date=21, t_before = 21, t_after = 164)        
        fin = model.output()
        
        # Step 4. Return just the soil moisture in a dataframe and append the simulation
        # number to x
        df['x_' + str(i)] = model.s
        
        # Step 5. Take the soil moisture of Dec. 31 and 
        # Re-run the simulation where the soil moisture on Jan 1st is the value that ended on Dec, 31. 
        sm0 = fin.s.tail(1)

    return df

# Step 6. Run this many times
result = initial_conditions(50)
result

In [ ]:
df = result

# Step 7. Remove the first year of data because that was an artificial soil moisture value. 
# So drop the first column (this needs to be out of the loop)
df = df.drop(['x_0'], axis=1)
    
# Step 8. End goal: For a given day of the year, return the mean and 
# variance of the soil moisture for that/each day. Plot the results.
df = result
df['mean'] = df.mean(axis=1)
df['var'] = df.var(axis=1)
df

# This is for one climatology and one maize variety (180 days)
plt.plot(df['mean'], 'k-')
x = np.arange(1, 366, 1) #x = np.arange(0, 208, 1) #np.arange(90, 135, 1)

plt.fill_between(x, df['var']+df['mean'], df['mean']-df['var'],facecolor='lightblue') #, facecolor='lightsalmon'
plt.title('Annual soil moisture for 180 day variety \n Planting date is Jan 1')
plt.ylabel('Relative soil moisture content, $\mathit{s}$')
plt.xlabel('Day of year, $\mathit{d}$')
plt.legend(['Mean', 'Variance'])
#plt.ylim(0.52, 0.62)

In [ ]:
# make crazy plot

sim = 10
    
df = pd.DataFrame({'SIM':range(0,sim)})
    
fig = plt.figure()

for i, sm0 in enumerate(np.linspace(0,1.0,11)):
    # Run model 
    result = initial_conditions(sim, sm0)
    # Extract SM values for first day
    
    doy = result.iloc[0,:].reset_index(drop=True)
    df['SM_' + str(i)] = doy
    
    # Plot
    plt.plot(df.SIM,doy)
    
plt.xlabel('Simulation')
plt.ylabel('Soil moisture')
plt.legend(labels=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],bbox_to_anchor=(1.02,1), loc="upper left")
    

## Plot Growing Season Soil Moisture Availability

Why are we doing this? Once the correct initial soil moisture has been found for a particular soil type and climate, then use it to generate a time series. 

Notes on parameters chosen:

* MAM planting date: Julian day 100 (early April) 
* Rainfall climatology:  Used Ol Jogi rainfall conditions 
* Question about this: A rainfall climatology for a year or the summary for a station?

Steps 1-8 same as above.

In [ ]:
# Generate soil moisture for the whole year
def annual_sm(simulations, sm0=0.3):
    
    # generate little progress bar
    import time
    import progressbar
    
    for i in progressbar.progressbar(range(simulations)):
        time.sleep(0.02)
    
    df = pd.DataFrame()
    alpha_r, lambda_r = make_climate_parameters()

    planting_date = 200
    
    # run this many times
    for i in range(simulations):
        
        # Step 1.  Start DOY before the planting season. 
        # Pick a day when there isn’t much rain: July 1, Julian Day 182 in non-leap year.

        # Step 2. Run t_sim for 365 so that calendar ends on June 30.
        climate = Climate(alpha_r, lambda_r)
        soil = Soil('loam')
        crop = Crop(soil=soil)
        soil.set_nZr(crop)  
        model = CropModel(crop=crop,soil=soil,climate=climate)

        model.run(planting_date=planting_date)

        # Step 3. Start planting date on some reasonable planting date
        # Picked early April Julian Day 100 in regular year. 
        model = CropModel(crop=crop,soil=soil,climate=climate)
        model.run(s0=sm0, planting_date=planting_date)
        fin = model.output()

        # Step 4
        df['x_' + str(i)] = fin.s
        
        # Step 5
        sm0 = fin.s.tail(1)

    return df 

result = annual_sm(50)
result

# Step 7. Remove the first year of data because that was an artificial soil moisture value. 
# So drop the first column (this needs to be out of the loop)

result = result.drop(['x_0'], axis=1)

# Step 8. End goal: For a given day of the year, return the mean and 
# variance of the soil moisture for that/each day. Plot the results.

df = result
df['mean'] = df.mean(axis=1)
df['var'] = df.var(axis=1)
df.to_csv('~/Desktop/200d.csv', sep=',')

# This is for one climatology and one maize variety (180 days)
plt.plot(df['mean'], 'k-')
x = np.arange(0, 208, 1) #np.arange(90, 135, 1)

plt.fill_between(x, df['mean']+df['var'], df['mean']-df['var'],facecolor='lightblue') #, facecolor='lightblue'
plt.title('Annual soil moisture for 180 day variety \n Planted on Julian Day 100')
plt.ylabel('Relative soil moisture content, $\mathit{s}$')
plt.xlabel('Day of season, $\mathit{d}$')
plt.legend(['Mean', 'Variance'])
#plt.ylim(0.52, 0.62)

In [ ]:
# Get some distributions

# Plot PDFs of growing season soil moisture

d90 = pd.read_csv('~/Desktop/100d.csv', sep=','); d90
d120 = pd.read_csv('~/Desktop/2d.csv', sep=',')
d180 = pd.read_csv('~/Desktop/200d.csv', sep=',')

d90['mean'].plot.kde(color='gray', linestyle='dashed', fillstyle='full',label='PD=100')

d120['mean'].plot.kde(color='lightblue', linestyle='dashdot', label='PD=2')
d180['mean'].plot.kde(label='PD=200')
plt.legend()
plt.ylabel('PDF')
plt.xlabel('Relative soil moisture content, $\mathit{s}$')
#plt.savefig('../output/figures/AvgGseasSM_KDE.pdf', format='pdf')


In [ ]:
alpha_r, lambda_r = make_climate_parameters()

# Generate soil moisture during LGP
def lgp_sm(simulations, tstart = 0.5, tend = 0.75):
    # generate little progress bar
    import time
    import progressbar
    
    for i in progressbar.progressbar(range(simulations)):
        time.sleep(0.02)
    
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    result = []
    
    # run this many times
    for i in range(simulations):
        
        # Step 1.  Start DOY before the planting season. 
        # Pick a day when there isn’t much rain: July 1, Julian Day 182 in non-leap year.

        # Step 2. Run t_sim for 365 so that calendar ends on June 30.
        climate = Climate(alpha_r, lambda_r)
        soil = Soil('loam')
        crop = Crop(soil=soil)
        soil.set_nZr(crop)  
        model = CropModel(crop=crop,soil=soil,climate=climate)

        # Step 3. Start planting date on some reasonable planting date
        # Picked early April Julian Day 100 in regular year. 
        model.run(planting_date=100) 
        fin = model.output()
        
        # Step 4
        df['x_' + str(i)] = fin.s
        
        # Step 5
        sm0 = fin.s.tail(1)
        
    return df

result = lgp_sm(50)
result

# Step 7. Remove the first year of data because that was an artificial soil moisture value. 
# So drop the first column (this needs to be out of the loop)
result = result.drop(['x_0'], axis=1)

df = result

# df = result.iloc[ 100:280 , : ] # something with this is funky
df['mean'] = df.mean(axis=1)
df['var'] = df.var(axis=1)
df

# Plot some stuff
# This is for one climatology and one maize variety (180 days)
plt.plot(df['mean'], 'k-')
x = np.arange(0, 208, 1) #np.arange(100, 280, 1) #np.arange(90, 135, 1)

plt.fill_between(x, df['var']+df['mean'], df['mean']-df['var'],facecolor='lightblue') #, facecolor='lightblue'
plt.title('LGP soil moisture: 180 day variety')
plt.ylabel('Relative soil moisture content, $\mathit{s}$')
plt.xlabel('Day of season, $\mathit{d}$')
plt.legend(['Mean', 'Variance'])


In [ ]:
# Generate soil moisture during plateau of crop coefficient (reproductive period)

def rep_sm(simulations, tstart = 0.5, tend = 0.75):
    # generate little progress bar
    import time
    import progressbar
    
    for i in progressbar.progressbar(range(simulations)):
        time.sleep(0.02)
    
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    result = []
    
    tseas =  120
    plantd = 100
    
    # run this many times
    for i in range(simulations):
        
        # Step 1.  Start DOY before the planting season. 
        # Pick a day when there isn’t much rain: July 1, Julian Day 182 in non-leap year.

        # Step 2. Run t_sim for 365 so that calendar ends on June 30.
        new_climate = {
            'alpha_r': [9.247170,9.989583,10.586111,11.529771,9.157616,10.351128,10.967919,9.553691,9.254545,8.326368,8.137118,6.693333], 
            'lambda_r': [0.051808,0.051502,0.105572,0.272917,0.304435,0.138542,0.174395,0.150202,0.114583,0.209157,0.246237,0.093652], 
            'doy_start': 182,
            't_sim': 365,
            'ET_max': 6.5
        } 
        climate = Climate(climate_parameters=new_climate) 
        soil = Soil('loam')
        crop = Crop(soil=soil)
        soil.set_nZr(crop)  

        # Step 3. Start planting date on some reasonable planting date
        # Picked early April Julian Day 100 in regular year. 
        model = CropModel(crop=crop,soil=soil,climate=climate, planting_date=100)
        model.run()
        fin = model.output()
        fin

        # Step 4. Take the soil moisture of June 30 and 
        # Re-run the simulation where the soil moisture on July 1st is the value that ended on June 30. 
        #print(fin.s.tail(1)) # The argument in tail is the last n lines
        last_s = fin.s.tail(1)

        # Step 5. Re-run the simulation where the soil moisture on July 1st is the value that ended on June 30. 
        # To do this we want to set s0 in model.run as last_s
        model.run(s0=last_s)
        fin = model.output()
        fin
        
        # Step 6. Run this lots and lots of times and return a matrix where the
        # row is each simulation and the column is doy
        
        # Return just the soil moisture in a dataframe and append the simulation
        # number to x
        df['x_' + str(i)] = model.s
        
    # Step 7. Remove the first year of data because that was an artificial soil moisture value. 
    # So drop the first column (this needs to be out of the loop)
    df = df.drop(['x_0'], axis=1)
    
    # Add the code back in with start and end that selects the rows we want
    # just select the part of the growing season that we want

    # Select certain rows; need to account for the fact that there's a planting date
    start = int(tseas*tstart+plantd)
    end = int(tseas*tend+plantd)
    print(start)
    print(end)
    df_ = df.iloc[ start:end , : ]
        
    # Take average soil moisture across each day
    df_['mean'] = df_.mean(axis=1)
    df_['var'] = df_.var(axis=1)
    
    return df_, start, end

df, start, end = rep_sm(1000)
df

# Plot some stuff
# This is for one climatology and one maize variety (180 days)
plt.plot(df['mean'], 'k-')
x = np.arange(start, end, 1) 

plt.fill_between(x, df['var']+df['mean'], df['mean']-df['var'],facecolor='orange') #, facecolor='lightblue', orange
plt.title('Reproductive period soil moisture: 120 day variety')
plt.ylabel('Relative soil moisture content, $\mathit{s}$')
plt.xlabel('Day of season, $\mathit{d}$')
plt.legend(['Mean', 'Variance'])
plt.ylim(0.37, 0.55)

plt.savefig('../output/figures/AvgRepSM_120d.pdf', format='pdf')

# Save the data as a csv
df.to_csv('../output/dataframes/AvgRepSM_120d.csv', sep=',')

## Old Functions

In [ ]:
# Average soil moisture during interval of the season
# What this is doing is basically the same as my code before and just returning 
# the average soil moisture values during the "growing" part of the season. 

def avg_sm_gseason(simulations, tstart = 0.5, tend = 0.75):
    result = []

    climate = Climate() 
    soil = Soil('loam')
    crop = Crop(soil=soil)
    soil.set_nZr(crop)  
    model = CropModel(crop=crop,soil=soil,climate=climate)

    model.run()
    fin = model.output()

    for i in range(simulations):
        # This part is not really necessary
        # Tseas is 180 which is just what the model is set to 
        # create variable for DOY
        tseas = 120
        range_tseas = list(range(1,181,1))

        #print(model.s)
        fin['doy'] = range_tseas
        len(fin.doy)

        # Select certain rows
        start = int(tseas*tstart)
        end = int(tseas*tend)
        x = fin.s[start:end]
      
        # Add this value to a list
        result.append(x)
        
    # Make it a float
    return np.array(result, dtype=np.float32)

test = avg_sm_gseason(1000)

test.mean(), test.var()

In [ ]:
def dist_avg_sm_gseason_daily(simulations, tstart = 0.5, tend = 0.75):
    # generate little progress bar
    import time
    import progressbar
    
    for i in progressbar.progressbar(range(simulations)):
        time.sleep(0.02)
    
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    result = []
    
    # run this many times
    for i in range(simulations):
        
        # initialize objects
        climate = Climate() 
        soil = Soil('loam')
        crop = Crop(soil=soil)
        soil.set_nZr(crop)  
        model = CropModel(crop=crop,soil=soil,climate=climate)
        model.run()
        fin = model.output()
        
        # Return just the soil moisture in a dataframe
        df['x_' + str(i)] = model.s
        
        # just select the part of the growing season that we want
        tseas = 180

        # Select certain rows
        start = int(tseas*tstart)
        end = int(tseas*tend)
        _df = df[start:end]
        
        # Take average soil moisture across each day
        df2['mean'] = _df.mean(axis=1)
        df2['var'] = _df.var(axis=1)

    return _df, df2 #result

df, df2 = dist_avg_sm_gseason_daily(100)

In [ ]:
# Plot some stuff
# This is for one climatology and one maize variety (180 days)
plt.plot(df2['mean'], 'k-')
x = np.arange(60, 90, 1) #np.arange(90, 135, 1)

plt.fill_between(x, df2['var']+df2['mean'], df2['mean']-df2['var'],facecolor='lightblue') #, facecolor='lightblue'
plt.title('Growing season soil moisture: 120 day variety')
plt.ylabel('Relative soil moisture content, $\mathit{s}$')
plt.xlabel('Day of season, $\mathit{d}$')
plt.legend(['Mean', 'Variance'])
plt.ylim(0.46, 0.51)

# I'm not sure that this is correct but it's somethingt

plt.savefig('../output/figures/AvgGseasSM_120d.pdf', format='pdf')

# Save the data as a csv
df2.to_csv('../output/dataframes/AvgGseasSM_120d.csv', sep=',')
# 

In [ ]:
def daily_sm(simulations):
    
    # generate little progress bar
    import time
    import progressbar
    
    for i in progressbar.progressbar(range(simulations)):
        time.sleep(0.02)
    
    df = pd.DataFrame()
    result = []
    
    # run this many times
    for i in range(simulations):
        
        # initialize objects
        climate = Climate() 
        soil = Soil('loam')
        crop = Crop(soil=soil)
        soil.set_nZr(crop)  
        model = CropModel(crop=crop,soil=soil,climate=climate)
        model.run()

        # Return just the soil moisture in a dataframe
        df['x_' + str(i)] = model.s
        
        # Take average soil moisture across each day
        df['mean'] = df.mean(axis=1)
        df['var'] = df.var(axis=1)

    return df #result

fin = daily_sm(10)

In [ ]:
fin

In [ ]:
plt.plot(fin['mean'], 'k-')
x = np.arange(0.0, 180, 1)

plt.fill_between(x, fin['var']+fin['mean'], fin['mean']-fin['var'])
                 #v facecolor='red'
# I'm not sure that this is correct but it's somethingt

In [ ]:
def average_soil_moisture(simulations):
    
    # generate little progress bar
    import time
    import progressbar
    
    for i in progressbar.progressbar(range(simulations)):
        time.sleep(0.02)
    
    result = []
    
    # run this many times
    for i in range(simulations):
        
        # initialize objects
        climate = Climate() 
        soil = Soil('loam')
        crop = Crop(soil=soil)
        soil.set_nZr(crop)  
        model = CropModel(crop=crop,soil=soil,climate=climate)

        model.run()
        model.output()

        # Return average soil moisture
        x = model.s.mean()

        # Add this value to a list
        result.append(x)
        
    return result

# TODO: One enhancement
# Suppress done!

In [ ]:
fin = average_soil_moisture(simulations=1000)

In [ ]:
# This is the average and variance of soil moisture for the 10000s of simulations
import statistics
statistics.mean(fin)

## Old code

In [ ]:
## Move this code elsewhere!
# example list of lists
import pandas as pd
import numpy as np
import glob

df1 = pd.DataFrame(np.random.randint(0,1000,size=(1500, 6)), columns=list('ABCDEF'))
df2 = pd.DataFrame(np.random.randint(0,1000,size=(1500, 6)), columns=list('ABCDEF'))
df3 = pd.DataFrame(np.random.randint(0,1000,size=(1500, 6)), columns=list('ABCDEF'))

output = [df1, df2, df3]
output

no_dfs = np.arange(0,len(output))

for i in no_dfs:
    output[i][start:end].to_csv('../output/final/%s.csv' % i)
    

In [ ]:
# how to read it back in

files = glob.glob('../output/final/*.csv')
output = []
for f, n in zip(files, no_dfs):
    f_ = pd.read_csv(f)
    output.append(f_)

output[0]

In [ ]:
plot.Cycle(dashes=[(1, 0.5), (1, 1.5)])

# make one graph of relative saturaiton with annotations
f, axs = plot.subplots(nrows = 1, ncols = 1, share=0) # , share=0, spanx=1, sharey=0
axs.format(
    abc=True, abcloc='ul', suptitle='Relative Saturation')#,

vwc = output[0]['s']
# convert s_star and sw
_sstar = crop.s_star
_sw = crop.sw

#plt.axhline(y=_sw, color='r', linestyle='--')
#plt.axhline(y=_sstar, color='r', linestyle='-')
xs = np.linspace(1,len(output[0]),len(output[0]))
horiz_line_data = np.array([_sw for i in range(len(xs))])
axs.plot(xs, horiz_line_data, 'r--') 
horiz_line_data = np.array([_sstar for i in range(len(xs))])
axs.plot(xs, horiz_line_data, 'r--') 

hs = axs.plot(vwc, color='k', lw=1, legend_kw={'order': 'F', 'title': 'column major'}) #color='lightblue',lw=1, 
axs.format(ylabel='Relative saturation')
axs.format(xlabel='Day of season')

# add legend
axs.legend(hs, loc='ll', label='legend label')